In [1]:
#import modules
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

pd.options.display.max_columns=999
pd.options.display.max_rows=3000

In [2]:

from openpyxl.workbook import Workbook


In [48]:
data = pd.read_excel('Collections Case Study Data.xlsx', sheet_name=None)

## What is the account roll forward rate for the months of Mar , Apr, May and June for TB0?

In [58]:
data.keys()

dict_keys(['Loan Details', 'Loan_Status_MartoMay', 'Loan_Status_AprtoJun', 'Historical 6 Months Details', 'Loan_ID mapping', 'Call Details'])

In [60]:
df = data

In [61]:
# start_data = df['Loan Details']
mar1_data = df['Loan_Status_MartoMay'].loc[df['Loan_Status_MartoMay']['Month'] == 'March']
apr1_data = df['Loan_Status_MartoMay'].loc[df['Loan_Status_MartoMay']['Month'] == 'April']
may1_data = df['Loan_Status_MartoMay'].loc[df['Loan_Status_MartoMay']['Month'] == 'May']
apr2_data = df['Loan_Status_AprtoJun'].loc[df['Loan_Status_AprtoJun']['Month'] == 'April']
may2_data = df['Loan_Status_AprtoJun'].loc[df['Loan_Status_AprtoJun']['Month'] == 'May']
jun2_data = df['Loan_Status_AprtoJun'].loc[df['Loan_Status_AprtoJun']['Month'] == 'June']

In [70]:
mar_data = pd.merge(mar1_data, apr2_data, how='left', on='Loan_id', suffixes=('_mar', '_marapr'))
apr_data = pd.merge(apr1_data, may2_data, how='left', on='Loan_id', suffixes=('_apr', '_aprmay'))
may_data = pd.merge(may1_data, jun2_data, how='left', on='Loan_id', suffixes=('_may', '_mayjun'))

In [71]:
may_data.head()

,Loan_id,Bucket_may,TENURE_may,PRINBALANCE_may,Months on Books_may,Sanctioned Amount_may,Month_may,Unnamed: 7_may,Unnamed: 8_may,Bucket_mayjun,PRINBALANCE_mayjun,TENURE_mayjun,Months on Books_mayjun,Sanctioned Amount_mayjun,Month_mayjun,Unnamed: 7_mayjun,Unnamed: 8_mayjun
0,A999990339,TB0,48,68114900.0,6,73000000,May,NaN,NaN,REGULAR,66303700.0,48,7,73000000,June,NaN,NaN
1,A999971854,TB0,48,51698000.0,24,80000000,May,NaN,NaN,REGULAR,48509900.0,48,25,80000000,June,NaN,-19605000.0
2,A999952066,TB0,36,35274500.0,14,50000000,May,NaN,NaN,REGULAR,32791800.0,36,15,50000000,June,NaN,NaN
3,A999938479,TB0,36,62723900.0,4,67000000,May,NaN,NaN,REGULAR,60348600.0,36,5,67000000,June,NaN,NaN
4,A999937745,TB0,36,44099400.0,5,48000000,May,NaN,NaN,REGULAR,42353900.0,36,6,48000000,June,NaN,NaN


In [85]:
data1 = pd.merge(mar_data, apr_data, how='outer', on='Loan_id')
data1 = pd.merge(data1, may_data, how='outer', on='Loan_id')

In [78]:
# dfs = [apr_data, may_data]
# for df in dfs:
#     data1 = pd.merge(mar_data, df, how='outer', on='Loan_id')

In [86]:
data1.head()

,Loan_id,Bucket_mar,TENURE_mar,PRINBALANCE_mar,Months on Books_mar,Sanctioned Amount_mar,Month_mar,Unnamed: 7_mar,Unnamed: 8_mar,Bucket_marapr,PRINBALANCE_marapr,TENURE_marapr,Months on Books_marapr,Sanctioned Amount_marapr,Month_marapr,Unnamed: 7_marapr,Unnamed: 8_marapr,Bucket_apr,TENURE_apr,PRINBALANCE_apr,Months on Books_apr,Sanctioned Amount_apr,Month_apr,Unnamed: 7_apr,Unnamed: 8_apr,Bucket_aprmay,PRINBALANCE_aprmay,TENURE_aprmay,Months on Books_aprmay,Sanctioned Amount_aprmay,Month_aprmay,Unnamed: 7_aprmay,Unnamed: 8_aprmay,Bucket_may,TENURE_may,PRINBALANCE_may,Months on Books_may,Sanctioned Amount_may,Month_may,Unnamed: 7_may,Unnamed: 8_may,Bucket_mayjun,PRINBALANCE_mayjun,TENURE_mayjun,Months on Books_mayjun,Sanctioned Amount_mayjun,Month_mayjun,Unnamed: 7_mayjun,Unnamed: 8_mayjun
0,A999993387,TB0,36.0,14361500.0,17.0,22000000.0,March,NaN,576300.0,REGULAR,13192100.0,36.0,18.0,22000000.0,April,NaN,609100.0,TB0,36.0,13785200.0,18.0,22000000.0,April,NaN,NaN,REGULAR,12583000.0,36.0,19.0,22000000.0,May,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A999990339,TB0,48.0,69805500.0,4.0,73000000.0,March,NaN,848000.0,REGULAR,68114900.0,48.0,5.0,73000000.0,April,NaN,NaN,TB0,48.0,68957500.0,5.0,73000000.0,April,NaN,842600.0,REGULAR,67206100.0,48.0,6.0,73000000.0,May,NaN,NaN,TB0,48.0,68114900.0,6.0,73000000.0,May,NaN,NaN,REGULAR,66303700.0,48.0,7.0,73000000.0,June,NaN,NaN
2,A999952066,TB0,36.0,37728800.0,12.0,50000000.0,March,NaN,1237600.0,REGULAR,35274500.0,36.0,13.0,50000000.0,April,NaN,NaN,TB0,36.0,36491200.0,13.0,50000000.0,April,NaN,1216700.0,REGULAR,34047600.0,36.0,14.0,50000000.0,May,NaN,NaN,TB0,36.0,35274500.0,14.0,50000000.0,May,NaN,NaN,REGULAR,32791800.0,36.0,15.0,50000000.0,June,NaN,NaN
3,A999937745,TB0,36.0,45753500.0,3.0,48000000.0,March,NaN,NaN,REGULAR,44099400.0,36.0,4.0,48000000.0,April,NaN,NaN,TB0,36.0,44923400.0,4.0,48000000.0,April,NaN,NaN,REGULAR,43235100.0,36.0,5.0,48000000.0,May,NaN,NaN,TB0,36.0,44099400.0,5.0,48000000.0,May,NaN,NaN,REGULAR,42353900.0,36.0,6.0,48000000.0,June,NaN,NaN
4,A999873098,TB0,48.0,42680200.0,12.0,50000000.0,March,NaN,NaN,REGULAR,41219400.0,48.0,13.0,50000000.0,April,NaN,NaN,TB0,48.0,41959400.0,13.0,50000000.0,April,NaN,NaN,REGULAR,40459600.0,48.0,14.0,50000000.0,May,NaN,NaN,TB0,48.0,41219400.0,14.0,50000000.0,May,NaN,NaN,REGULAR,39679600.0,48.0,15.0,50000000.0,June,NaN,NaN


In [87]:
data1.columns

Index(['Loan_id', 'Bucket_mar', 'TENURE_mar', 'PRINBALANCE_mar',
       'Months on Books_mar', 'Sanctioned Amount_mar', 'Month_mar',
       'Unnamed: 7_mar', 'Unnamed: 8_mar', 'Bucket_marapr',
       'PRINBALANCE_marapr', 'TENURE_marapr', 'Months on Books_marapr',
       'Sanctioned Amount_marapr', 'Month_marapr', 'Unnamed: 7_marapr',
       'Unnamed: 8_marapr', 'Bucket_apr', 'TENURE_apr', 'PRINBALANCE_apr',
       'Months on Books_apr', 'Sanctioned Amount_apr', 'Month_apr',
       'Unnamed: 7_apr', 'Unnamed: 8_apr', 'Bucket_aprmay',
       'PRINBALANCE_aprmay', 'TENURE_aprmay', 'Months on Books_aprmay',
       'Sanctioned Amount_aprmay', 'Month_aprmay', 'Unnamed: 7_aprmay',
       'Unnamed: 8_aprmay', 'Bucket_may', 'TENURE_may', 'PRINBALANCE_may',
       'Months on Books_may', 'Sanctioned Amount_may', 'Month_may',
       'Unnamed: 7_may', 'Unnamed: 8_may', 'Bucket_mayjun',
       'PRINBALANCE_mayjun', 'TENURE_mayjun', 'Months on Books_mayjun',
       'Sanctioned Amount_mayjun', 'Month

In [64]:
data1['Bucket_y'].value_counts()

REGULAR    22475
TB0         1709
TB1          391
Name: Bucket_y, dtype: int64

In [89]:
cols = ['Bucket_mar','Bucket_marapr', 'Bucket_apr', 'Bucket_aprmay', 'Bucket_may', 'Bucket_mayjun']

for c in cols:
    print(data1[c].value_counts())
    print('\n')

TB0    24575
Name: Bucket_mar, dtype: int64


REGULAR    22475
TB0         1709
TB1          391
Name: Bucket_marapr, dtype: int64


TB0    27079
Name: Bucket_apr, dtype: int64


REGULAR    24964
TB0         1574
TB1          541
Name: Bucket_aprmay, dtype: int64


TB0    29115
Name: Bucket_may, dtype: int64


REGULAR    27085
TB0         1577
TB1          453
Name: Bucket_mayjun, dtype: int64




In [121]:
data1.shape

(40339, 49)

In [93]:
exp = data1['Bucket_marapr'].value_counts().to_frame()
exp

,Bucket_marapr
REGULAR,22475
TB0,1709
TB1,391


In [107]:
cols1 = ['Bucket_marapr', 'Bucket_aprmay', 'Bucket_mayjun']
exp1 = pd.DataFrame()
for c in cols1:
    exp1[c] = data1[c].value_counts()


In [108]:
exp1

,Bucket_marapr,Bucket_aprmay,Bucket_mayjun
REGULAR,22475,24964,27085
TB0,1709,1574,1577
TB1,391,541,453


In [110]:
exp1.iloc[2,0]

391

In [122]:
mar_roll = round(exp1.iloc[2,0] / exp1['Bucket_marapr'].sum() * 100, 3)
apr_roll = round(exp1.iloc[2,1] / exp1['Bucket_aprmay'].sum() * 100, 3)
may_roll = round(exp1.iloc[2,2] / exp1['Bucket_mayjun'].sum() * 100, 3)

In [123]:
print(mar_roll)
print(apr_roll)
print(may_roll)


1.591
1.998
1.556


## What is the average attempt intensity, RPC rate and PTP rate on accounts for the months of mar, Apr and May

In [4]:
df.keys()

dict_keys(['Loan Details', 'Loan_Status_MartoMay', 'Loan_Status_AprtoJun', 'Historical 6 Months Details', 'Loan_ID mapping', 'Call Details'])

In [5]:
call_data = df['Call Details']
call_data.head()

,month,Application_Id,Login_ID,Right_Party_Contact,Promise_to_pay,total_contacts
0,3,MX-100062359,AGNT50,1,1,1
1,3,MX-100095892,AGNT12,1,0,2
2,3,MX-100116696,AGNT36,2,0,3
3,3,MX-10011780,AGNT39,2,1,10
4,3,MX-100133826,AGNT48,4,3,17


In [11]:
call_data_table = call_data.groupby(['month']).sum()

In [15]:
call_data_table['RPC_rate'] = round((call_data['Right_Party_Contact'] / call_data['total_contacts']) * 100, 3)

In [16]:
call_data_table.head()

,Right_Party_Contact,Promise_to_pay,total_contacts,RPC_rate
month,,,,
3,82627,29693,300513,20.000
4,84211,29784,282200,23.529
5,89033,29625,292643,50.000


In [18]:
call_data_table['PTP_rate'] = round((call_data['Promise_to_pay'] / call_data['total_contacts']) * 100, 3)

In [21]:
call_data_table

,Right_Party_Contact,Promise_to_pay,total_contacts,RPC_rate,PTP_rate
month,,,,,
3,82627,29693,300513,20.000,10.000
4,84211,29784,282200,23.529,17.647
5,89033,29625,292643,50.000,0.000
